# Proposal - MLND Capstone Project
### Chad Acklin
***


## Data Acquisition
As noted in the project proposal, due to the privacy concerns of real-world personally identifiable datasets, this project will make use of synthetically generated data.  The initial dataset was generated with the generate tool published by the [Freely Extensible Biomedical Record Linkage](http://users.cecs.anu.edu.au/~Peter.Christen/Febrl/febrl-0.3/febrldoc-0.3/node70.html) package.  The FEBRL tool generates a dataset of demographic data and introduces noise that mimics OCR errors, typograhic errors, and phonetic misspellings.  

A dataset of 30,000 originals and 15,000 matches was generated, stored as a csv, and loaded below.  Originals have an id of rec-X-org and matches have an id of rec-X-dup-X

In [1]:
import pandas as pd
import numpy as np
import jellyfish
import matplotlib.pyplot as plt
%matplotlib inline
from time import time

In [2]:
col = ['id', 'nationality', 'gender', 'age', 'dob', 'title',
       'first_name', 'last_name', 'state', 'city', 'zip',
       'street_number', 'address_1', 'address_2', 'phone']
df = pd.read_csv('FEBRL_sample_data.csv', encoding = 'latin1', names = col)
df.head()

,id,nationality,gender,age,dob,title,first_name,last_name,state,city,zip,street_number,address_1,address_2,phone
0,rec_id,culture,sex,age,date_of_birth,title,given_name,surname,state,suburb,postcode,street_number,address_1,address_2,phone_number
1,rec-0-dup-0,NaN,f,28,19901218,,caitlin,chappel,,pres,4504,NaN,g'lbbes street,,087631 3909
2,rec-0-dup-1,NaN,f,2,19901218,,cait in,chap pl,,preston,4504,NaN,gibbess reet,,087631 909
3,rec-0-org,NaN,f,28,19901218,,caitlin,chappel,,preston,4504,NaN,gibbes street,,08 76313909
4,rec-1-org,ind,f,,19630324,,oscar,murjani,vic,bright,2060,14,aspinall street,,618 53569883



After loading the dataset, several additional features are calculated including phonetic representations of first name, last name, and city.  These were created using the metaphone algorithm available in the jellyfish package

Data is into 2 dataframes by originals and duplicates.

In [3]:
df[['prefix','trueRecordID', 'OrgDup', 'seqNum']] = df.id.str.split('-', expand=True)
df.drop('prefix', axis=1, inplace=True)
df.fillna ('', inplace=True)

In [4]:
df['stringConcat'] = df.title+df.first_name+df.last_name+df.city+df.state+df.zip+df.address_1+df.address_2+df.phone

In [5]:
df['last_name_met'] = df['last_name'].str.replace(' ', '').apply(lambda x: jellyfish.metaphone(x))
df['first_name_met'] = df['first_name'].str.replace(' ', '').apply(lambda x: jellyfish.metaphone(x))
df['city_met'] = df['city'].str.replace(' ', '').apply(lambda x: jellyfish.metaphone(x))
df['phone'] = df['phone'].str.replace(' ', '')
df['predicateLastName'] = df['last_name'].str[:2]
df['predicateFirstName'] = df['first_name'].str[:2]


dfDup = df[df.OrgDup == 'dup']
dfOrg = df[df.OrgDup == 'org']
dfDup.columns = [str(col) + '_match' for col in dfDup.columns]
dfOrg.columns = [str(col) + '_org' for col in dfOrg.columns]
print('Original Recordset:', dfOrg.shape)
print('Recordset of matches:', dfDup.shape)

Original Recordset: (30000, 24)
Recordset of matches: (15000, 24)


## Blocking
### Traditional Approach

The demographics are used to "block" potential mathes between the datasets.  Some trial and error was used here to find a balance of limiting the dataset and the completeness of including all matching pairs.  If every original were compared to every potential match, the resultant dataset would be 450M rows (30,000 x 15,000).  Blocking limits the number of comparisons that will be required.

After blocking, the paired dataset includes slightly more than 4.18M rows and has only failed to include 11 matches.  This is an acceptable trade-off for this task and the blocked matches becomes the basis for building the model.

In [6]:
t1 = time()

dfZipMatches = pd.merge(dfOrg, dfDup, how='inner', left_on='zip_org', right_on='zip_match')
dfSurnameMatches = pd.merge(dfOrg, dfDup, how='inner', left_on='last_name_met_org', right_on='last_name_met_match')
dfCityMatches = pd.merge(dfOrg, dfDup, how='inner', left_on='city_met_org', right_on='city_met_match')
dfPhoneMatches = pd.merge(dfOrg, dfDup, how='inner', left_on='phone_org', right_on='phone_match')
dfPredicateMatches = pd.merge(dfOrg, dfDup, how='inner'
                                , left_on=['predicateFirstName_org', 'predicateLastName_org']
                                , right_on=['predicateFirstName_match', 'predicateLastName_match'])

In [7]:
matchFrames = [ dfZipMatches
              , dfSurnameMatches
              , dfCityMatches
              , dfPhoneMatches
              , dfPredicateMatches
              ]

for m in matchFrames:
    print( m.shape)

(421896, 48)
(689347, 48)
(174402, 48)
(1196104, 48)
(1785915, 48)


In [8]:
dfBlockedMatches = pd.concat(matchFrames)

In [9]:
## Clean up a little and drop duplicates
dfBlockedMatches.fillna('0', inplace=True)
dfBlockedMatches.drop_duplicates(subset=['id_org', 'id_match'], inplace=True)

dfBlockedMatches['MATCH'] = np.where(dfBlockedMatches[
        'trueRecordID_org']==dfBlockedMatches['trueRecordID_match'], 1, 0)

t2 = time()
time_traditional_blocking = t2-t1

## How many of the 15,000 matches have we included in the dataset?
truePos_traditional = sum(dfBlockedMatches.MATCH)

print('Traditional Blocking result shape:', dfBlockedMatches.shape)
print('Traditional Blocking time:',time_traditional_blocking)
print('Traditional Blocking true positives included:',truePos_traditional)

Traditional Blocking result shape: (4183821, 49)
Traditional Blocking time: 14.165188789367676
Traditional Blocking true positives included: 14989


## A different path to blocking
### TF-IDF similarity as blocking criteria

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import numpy as np
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct

In [16]:
def ngrams(string, n=4):
    string = re.sub(r'[,-./]|\s',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

ngrams('John Doe Louisville KY 40202 555123657')

['John',
 'ohnD',
 'hnDo',
 'nDoe',
 'DoeL',
 'oeLo',
 'eLou',
 'Loui',
 'ouis',
 'uisv',
 'isvi',
 'svil',
 'vill',
 'ille',
 'lleK',
 'leKY',
 'eKY4',
 'KY40',
 'Y402',
 '4020',
 '0202',
 '2025',
 '0255',
 '2555',
 '5551',
 '5512',
 '5123',
 '1236',
 '2365',
 '3657']

In [17]:
def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))


In [18]:
t3 = time()

dfConcatStrings_org = dfOrg.stringConcat_org
dfConcatStrings_match = dfDup.stringConcat_match
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
vectorizer.fit(df.stringConcat)
tf_idf_matrix_org = vectorizer.transform(dfConcatStrings_org)
tf_idf_matrix_match = vectorizer.transform(dfConcatStrings_match)

matches = awesome_cossim_top(tf_idf_matrix_org, tf_idf_matrix_match.transpose(), 10, 0.1)

In [19]:
dfBlockedMatches = pd.DataFrame(np.column_stack(matches.nonzero()), columns = ['OriginalIndex', 'MatchIndex'])
dfBlockedMatches['similarity'] = matches.data
dfBlockedMatches = dfBlockedMatches.merge(dfOrg.reset_index(), left_on = 'OriginalIndex', right_index=True )
dfBlockedMatches = dfBlockedMatches.merge(dfDup.reset_index(), left_on = 'MatchIndex', right_index=True )

In [20]:
dfBlockedMatches.drop_duplicates(subset=['id_org', 'id_match'], inplace=True) 
dfBlockedMatches['MATCH'] = np.where(dfBlockedMatches[
        'trueRecordID_match']==dfBlockedMatches['trueRecordID_org'], 1, 0)


t4 = time()
time_TFIDF_Blocking = t4-t3

## How many of the 15,000 matches have we included in the dataset?
truePos_TfIdf = sum(dfBlockedMatches.MATCH)


print('TF-IDF Blocking result shape:', dfBlockedMatches.shape)
print('TF-IDF Blocking time:',time_TFIDF_Blocking)
print('TF-IDF Blocking true positives included:',truePos_TfIdf)

TF-IDF Blocking result shape: (175712, 54)
TF-IDF Blocking time: 5.727512836456299
TF-IDF Blocking true positives included: 14999


In [21]:
dfBlockedMatches.head().transpose()

,0,47132,63515,89764,1
OriginalIndex,0,8120,10893,15428,0
MatchIndex,0,0,0,0,1
similarity,0.683051,0.138749,0.113165,0.101841,0.652186
index_x,3,12146,16354,23030,3
id_org,rec-0-org,rec-17305-org,rec-19801-org,rec-23883-org,rec-0-org
nationality_org,,ind,chi,unk,
gender_org,f,,f,f,f
age_org,28,30,9,26,28
dob_org,19901218,19881105,20091030,19921204,19901218
title_org,,,ms,,


## Feature Engineering

Next, features are engineered that compare the edit distance of associated strings between the original and match.  This step uses the Damerau Levenshtein edit distance to determine the similarity of the strings.  Then, the edit distances are scaled by a factor of 10.

In [22]:
dfBlockedMatches['phone_dlev'] = dfBlockedMatches.apply(lambda x: jellyfish.damerau_levenshtein_distance
                                                        (x['phone_org'], x['phone_match']), axis=1)
dfBlockedMatches['last_name_dlev'] = dfBlockedMatches.apply(lambda x: jellyfish.damerau_levenshtein_distance
                                                        (x['last_name_org'], x['last_name_match']), axis=1)
dfBlockedMatches['first_name_dlev'] = dfBlockedMatches.apply(lambda x: jellyfish.damerau_levenshtein_distance
                                                        (x['first_name_org'], x['first_name_match']), axis=1)
dfBlockedMatches['city_dlev'] = dfBlockedMatches.apply(lambda x: jellyfish.damerau_levenshtein_distance
                                                        (x['city_org'], x['city_match']), axis=1)
dfBlockedMatches['state_dlev'] = dfBlockedMatches.apply(lambda x: jellyfish.damerau_levenshtein_distance
                                                        (x['state_org'], x['state_match']), axis=1)
dfBlockedMatches['address_1_dlev'] = dfBlockedMatches.apply(lambda x: jellyfish.damerau_levenshtein_distance
                                                        (x['address_1_org'], x['address_1_match']), axis=1)
dfBlockedMatches['address_1_2_dlev'] = dfBlockedMatches.apply(lambda x: jellyfish.damerau_levenshtein_distance
                                                        (x['address_1_org'], x['address_2_match']), axis=1)
dfBlockedMatches['dob_dlev'] = dfBlockedMatches.apply(lambda x: jellyfish.damerau_levenshtein_distance
                                                        (x['dob_org'], x['dob_match']), axis=1)
dfBlockedMatches['zip_dlev'] = dfBlockedMatches.apply(lambda x: jellyfish.damerau_levenshtein_distance
                                                        (x['zip_org'], x['zip_match']), axis=1)

In [29]:
def scale_invert(x):
    scaled = 1-(x/10)
    return max(0, scaled)

In [30]:
dfBlockedMatches['phone_scaled'] = dfBlockedMatches['phone_dlev'].apply(scale_invert)
dfBlockedMatches['last_name_scaled'] = dfBlockedMatches['last_name_dlev'].apply(scale_invert)
dfBlockedMatches['first_name_scaled'] = dfBlockedMatches['first_name_dlev'].apply(scale_invert)
dfBlockedMatches['city_scaled'] = dfBlockedMatches['city_dlev'].apply(scale_invert)
dfBlockedMatches['state_scaled'] = dfBlockedMatches['state_dlev'].apply(scale_invert)
dfBlockedMatches['address_1_scaled'] = dfBlockedMatches['address_1_dlev'].apply(scale_invert)
dfBlockedMatches['address_1_2_scaled'] = dfBlockedMatches['address_1_2_dlev'].apply(scale_invert)
dfBlockedMatches['dob_scaled'] = dfBlockedMatches['dob_dlev'].apply(scale_invert)
dfBlockedMatches['zip_scaled'] = dfBlockedMatches['zip_dlev'].apply(scale_invert)

In [31]:
dfBlockedMatches.columns

Index(['OriginalIndex', 'MatchIndex', 'similarity', 'index_x', 'id_org',
       'nationality_org', 'gender_org', 'age_org', 'dob_org', 'title_org',
       'first_name_org', 'last_name_org', 'state_org', 'city_org', 'zip_org',
       'street_number_org', 'address_1_org', 'address_2_org', 'phone_org',
       'trueRecordID_org', 'OrgDup_org', 'seqNum_org', 'stringConcat_org',
       'last_name_met_org', 'first_name_met_org', 'city_met_org',
       'predicateLastName_org', 'predicateFirstName_org', 'index_y',
       'id_match', 'nationality_match', 'gender_match', 'age_match',
       'dob_match', 'title_match', 'first_name_match', 'last_name_match',
       'state_match', 'city_match', 'zip_match', 'street_number_match',
       'address_1_match', 'address_2_match', 'phone_match',
       'trueRecordID_match', 'OrgDup_match', 'seqNum_match',
       'stringConcat_match', 'last_name_met_match', 'first_name_met_match',
       'city_met_match', 'predicateLastName_match', 'predicateFirstName_match

## Final Dataset

The final dataset is saved as csv and a sample generated to attach to the proposal.  The final dataset includes the original id, the id from the matching dataset, a "MATCH" indicator that will be our target variable, and scaled features based upon the string edit distances calculated above.  

In [32]:
dfFeatures = dfBlockedMatches[[ 'id_org', 'id_match', 'MATCH',
       'phone_scaled', 'last_name_scaled', 'first_name_scaled', 'city_scaled',
       'state_scaled', 'address_1_scaled', 'address_1_2_scaled', 'dob_scaled',
       'zip_scaled', 'similarity']]
dfFeatures.to_csv('dfFeatures.csv')
#dfFeatures.sample(n=10000).to_csv('dfFeatures_sample.csv')

In [33]:
dfFeatures.head()

,id_org,id_match,MATCH,phone_scaled,last_name_scaled,first_name_scaled,city_scaled,state_scaled,address_1_scaled,address_1_2_scaled,dob_scaled,zip_scaled,similarity
0,rec-0-org,rec-0-dup-0,1,1.0,1.0,1.0,0.7,1.0,0.8,0.0,1.0,1.0,0.683051
47132,rec-17305-org,rec-0-dup-0,0,0.0,0.8,1.0,0.2,0.8,0.0,0.0,0.5,0.7,0.138749
63515,rec-19801-org,rec-0-dup-0,0,0.0,0.4,0.4,0.4,0.7,0.3,0.0,0.4,0.7,0.113165
89764,rec-23883-org,rec-0-dup-0,0,0.4,0.3,0.5,0.2,0.8,0.3,0.0,0.7,0.6,0.101841
1,rec-0-org,rec-0-dup-1,1,0.9,0.8,0.9,1.0,1.0,0.8,0.0,1.0,1.0,0.652186


In [34]:
dfFeatures.shape

(175712, 13)